In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
cleaned_df = pd.read_csv("Resources/cleaned_up_dataset.csv")
cleaned_df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,hit,genre
0,4pR4oQJULf7FDB54TleMyH,Limoblaze;Lecrae;Happi,Jireh (My Provider),Jireh (My Provider),64,168000,False,0.443,0.778,0,...,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,1.0,Hip_Hop_R&B
1,4Ds4bq6aqOSAp1T7DikOi4,Criolo,Ainda Há Tempo,Ainda Há Tempo,44,318360,False,0.580,0.800,6,...,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,0.0,Hip_Hop_R&B
2,7nYbKsvhM88mHCmFsAIQVp,Criolo,Fellini,Fellini,42,171989,False,0.805,0.746,1,...,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,0.0,Hip_Hop_R&B
3,7gVNP7rI9UBZndge0ulKfL,Criolo,Nó na Orelha,Lion Man,46,205440,False,0.582,0.660,7,...,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,0.0,Hip_Hop_R&B
4,1hLvWelTny8vttEEZIXVjw,Criolo,Espiral de Ilusão,Espiral de Ilusão,44,220800,False,0.776,0.314,2,...,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,0.0,Hip_Hop_R&B


In [3]:
# Drops track_id, artists, album_name, track_name, popularity, explicit
cleaned_df.drop(['track_id','artists','album_name', 'track_name', 'popularity', 'explicit'], axis = 1, inplace=True)
cleaned_df.head()

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,hit,genre
0,168000,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,1.0,Hip_Hop_R&B
1,318360,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,0.0,Hip_Hop_R&B
2,171989,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,0.0,Hip_Hop_R&B
3,205440,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,0.0,Hip_Hop_R&B
4,220800,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,0.0,Hip_Hop_R&B


In [4]:
# Generate our categorical variable lists
cleaned_cat = cleaned_df.dtypes[cleaned_df.dtypes == 'object'].index.tolist()
cleaned_cat

['genre']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(cleaned_df[cleaned_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cleaned_cat)
encode_df.head()

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre_Cinema,genre_Classical,genre_Country,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Merge the dataframes and drop genre
cleaned_df = cleaned_df.merge(encode_df, left_index=True, right_index=True).drop("genre",1)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [7]:
cleaned_df.head()

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,168000,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,318360,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,171989,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,205440,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,220800,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = cleaned_df['hit']
X = cleaned_df.drop(['hit'],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.825


In [11]:
# Display sorted list of features and their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09443070781932422, 'duration_ms'),
 (0.0903456860484811, 'acousticness'),
 (0.09004574276071983, 'danceability'),
 (0.08874815907556055, 'valence'),
 (0.08867408883327382, 'loudness'),
 (0.0881600249070131, 'speechiness'),
 (0.08757349388872292, 'tempo'),
 (0.08547549037131887, 'energy'),
 (0.08261090307680845, 'liveness'),
 (0.0676280419330542, 'instrumentalness'),
 (0.04690237087896971, 'key'),
 (0.013715704644400892, 'genre_Pop'),
 (0.011345363331978407, 'mode'),
 (0.008324523434798153, 'genre_Latin'),
 (0.0077197810512862775, 'time_signature'),
 (0.0064208726354835054, 'genre_Rock'),
 (0.006320923429442003, 'genre_Dance'),
 (0.00544341454168894, 'genre_Foreign'),
 (0.004741567557095247, 'genre_Indie'),
 (0.004345774157453279, 'genre_Cinema'),
 (0.004315058014231942, 'genre_Classical'),
 (0.004102783505915175, 'genre_Kids'),
 (0.0037702356411788113, 'genre_Hip_Hop_R&B'),
 (0.003178806138208457, 'genre_Reggae'),
 (0.0031262993785709323, 'genre_Country'),
 (0.0025341829450213483, 